In [1]:
import numpy as np
import pandas as pd
from hyperopt import hp, fmin, tpe, space_eval

In [5]:
df = pd. read_csv("Employee_Salaries_-_2017.csv")
df.head()

,Full Name,Gender,Current Annual Salary,2017 Gross Pay Received,2017 Overtime Pay,Department,Department Name,Division,Assignment Category,Employee Position Title,Position Under-Filled,Date First Hired
0,"Aarhus, Pam J.",F,70959.79,71316.72,0.00,POL,Department of Police,MSB Information Mgmt and Tech Division Records...,Fulltime-Regular,Office Services Coordinator,NaN,09/22/1986
1,"Aaron, Marsha M.",F,110359.00,108040.82,0.00,HHS,Department of Health and Human Services,Adult Protective and Case Management Services,Fulltime-Regular,Supervisory Social Worker,NaN,11/19/1989
2,"Ababio, Godfred A.",M,55950.24,62575.19,7649.19,COR,Correction and Rehabilitation,PRRS Facility and Security,Fulltime-Regular,Resident Supervisor II,NaN,05/05/2014
3,"Ababu, Essayas",M,95740.00,96055.94,0.00,HCA,Department of Housing and Community Affairs,Affordable Housing Programs,Fulltime-Regular,Planning Specialist III,NaN,03/05/2007
4,"Abbamonte, Drew B.",M,74732.00,98736.78,23468.73,POL,Department of Police,PSB 6th District Special Assignment Team,Fulltime-Regular,Police Officer III,NaN,07/16/2007


In [7]:
len(df["Department Name"].unique())

37

In [9]:
# define an objective function
def objective(args):
    print(args)
    df_temp = df
    for col in args.keys():
        df_temp = df_temp.loc[df_temp[col] == args[col]]
    grp = df_temp.groupby(optimizing_dimension).mean()
    
    if 'F' in grp.index.values and 'M' in grp.index.values:
        return - grp.loc["M"][measure] / grp.loc["F"][measure]
    else:
        return 0


grouping_dimensions = ["Department Name", "Assignment Category"]
optimizing_dimension = ["Gender"]
measure = "Current Annual Salary"
df = df[grouping_dimensions + optimizing_dimension + [measure]]

constraints_dict = dict()
for col in grouping_dimensions:
    constraints_dict[col] = hp.choice(col, list(df[col].unique()))

# define a search space
space = hp.choice('range constraints', [constraints_dict])

# minimize the objective over the space
best = fmin(objective, space, algo=tpe.suggest, max_evals=50)

print(best)
print(space_eval(space, best))

{'Assignment Category': 'Fulltime-Regular', 'Department Name': 'Office of Consumer Protection'}
{'Assignment Category': 'Parttime-Regular', 'Department Name': 'Department of Transportation'}
{'Assignment Category': 'Parttime-Regular', 'Department Name': 'Office of Emergency Management and Homeland Security'}
{'Assignment Category': 'Fulltime-Regular', 'Department Name': 'Office of Emergency Management and Homeland Security'}
{'Assignment Category': 'Parttime-Regular', 'Department Name': 'Department of Housing and Community Affairs'}
{'Assignment Category': 'Parttime-Regular', 'Department Name': 'Merit System Protection Board Department'}
{'Assignment Category': 'Fulltime-Regular', 'Department Name': 'Department of General Services'}
{'Assignment Category': 'Parttime-Regular', 'Department Name': 'Department of Liquor Control'}
{'Assignment Category': 'Parttime-Regular', 'Department Name': 'Office of Consumer Protection'}
{'Assignment Category': 'Fulltime-Regular', 'Department Name': 'Of

In [46]:
mini = 0
name = ""
for val in list(df["Department Name"].unique()):
    df_temp = df.loc[df["Department Name"] == val]
    grp = df_temp.groupby("Gender").mean()
    if 'F' in grp.index.values and 'M' in grp.index.values:
        if - grp.loc["M"][measure] / grp.loc["F"][measure] < mini:
            mini = - grp.loc["M"][measure] / grp.loc["F"][measure]
            name = val
print(val)
print(mini)

Merit System Protection Board Department
-3.172759422576303
